<a href="https://colab.research.google.com/github/RogerHeederer/ML_practice_MyFirstML_B/blob/master/MLP_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

source reference : https://github.com/wikibook/machine-learnin

위 레퍼런스를 참조하여 공부하였으며, 필요하다고 생각되는 부분에 추가적인 설명 및 그림들을 덧붙였습니다.

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
tf.random.set_seed(678)

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Activation
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from IPython.display import Image

In [6]:
# MLP 다층퍼셉트론 구조
Image(url = "https://raw.githubusercontent.com/captainchargers/deeplearning/master/img/dropout.png", width=500, height=250)

In [8]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [9]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [42]:
x_train[59999]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [15]:
y_train[0:15]

array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 3, 5, 3, 6, 1], dtype=uint8)

In [23]:
print("train data has " + str(x_train.shape[0]) + "samples")

train data has 60000samples


In [24]:
print("every train data is "+ str(x_train.shape[1]) + " * " + str(x_train.shape[2]) + "image")

every train data is 28 * 28image


In [25]:
print("test data has " + str(x_test.shape[0]) + " samples")
print("every test data is " + str(x_test.shape[1]) 
      + " * " + str(x_test.shape[2]) + " image")

test data has 10000 samples
every test data is 28 * 28 image


In [28]:
#데이터 정규화 - 학습시간 단축 및 퍼포먼스 향상
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

gray_scale = 255
x_train /= gray_scale
x_test /= gray_scale

##**텐서플로우로 MLP 구현하기**

In [29]:
Image(url= "https://raw.githubusercontent.com/captainchargers/deeplearning/master/img/dropout.png", width=500, height=250)

In [30]:
# 다층퍼셉트론의 입력 레이어에 데이터를 넣기 위해서는 (28,28) 2d tensor를 1d 타입으로 바꿔줘야 한다
# 즉, 행렬 데이터 타입을 배열 데이터 타입으로 전환해줘야 함
Image(url= "https://raw.githubusercontent.com/captainchargers/deeplearning/master/img/reshape_mnist.png", width=300, height=150)

In [31]:
model = Sequential([
                    Flatten(input_shape=(28,28)),
                    Dense(256, activation='relu'), #첫번째 히든레이어
                    Dense(128, activation='relu'), #두번째 히든레이어
                    Dropout(0.1),#두번째 히든레이어에 드랍아웃 적용
                    Dense(10),# 세번째 히든 레이어
                    Activation('softmax')
])

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               200960    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
_________________________________________________________________
activation (Activation)      (None, 10)                0         
Total params: 235,146
Trainable params: 235,146
Non-trainable params: 0
__________________________________________________

- 인풋값 784 * 인풋을 받아주는 첫번째 레이어의 노드 수 256 + 노드마다의 편향값 256 = 200960
- 첫번째 레이어로부터 나온 노드 256 * 두번째 레이어의 노드 수 128 + 두번째 레이어의 노드마다 편향값 128 = 32896
- 두번째 레이어로부터 나온 노드 128 * 세번째 레이어의 노드 수 10 + 세번째 레이어의 노드마다 편향값 10 = 1290
- flatten과 softmax는 노드가 없으므로 파라미터 값은 0

In [35]:
#softmax를 통해 나온 예측값 y와 실측값 y_hat을 비교하여, 둘 간의 loss가 최소인 상태로 만들고자 한다
#그러기 위해 sparse_categorical_crossentopy를 사용하는데, 레이블 값을 원 핫 인코딩으로 자동 변경하여 크로스 엔트로피를 측정한다.
#그걸 adam 옵티마이져가 back propagation을 진행하면서 수행해준다.
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

###조기종료 (Early Stopping)
매 주기(epoch)마다 검증 데이터로 검증 정확도를 측정한다.
val_accuracy가 5번 연속으로 개선되지 않을 시, 조기에 훈련 종료를 한다

In [36]:
callbacks = [ EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=False),
              ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True)]

In [37]:
model.fit(x_train, y_train, epochs=300, batch_size=1000,
          validation_split = 0.1, callbacks=callbacks) # validation_split=0.1 은 6만개 데이터 중 54000개는 train, 6000개는 validation으로 분할

Epoch 1/300
54/54 [==============================] - 2s 28ms/step - loss: 0.6740 - accuracy: 0.8117 - val_loss: 0.2151 - val_accuracy: 0.9403
Epoch 2/300
54/54 [==============================] - 1s 25ms/step - loss: 0.2391 - accuracy: 0.9307 - val_loss: 0.1459 - val_accuracy: 0.9638
Epoch 3/300
54/54 [==============================] - 1s 26ms/step - loss: 0.1713 - accuracy: 0.9505 - val_loss: 0.1238 - val_accuracy: 0.9665
Epoch 4/300
54/54 [==============================] - 1s 25ms/step - loss: 0.1341 - accuracy: 0.9607 - val_loss: 0.1038 - val_accuracy: 0.9688
Epoch 5/300
54/54 [==============================] - 1s 25ms/step - loss: 0.1091 - accuracy: 0.9682 - val_loss: 0.0910 - val_accuracy: 0.9730
Epoch 6/300
54/54 [==============================] - 1s 25ms/step - loss: 0.0922 - accuracy: 0.9729 - val_loss: 0.0867 - val_accuracy: 0.9742
Epoch 7/300
54/54 [==============================] - 1s 26ms/step - loss: 0.0770 - accuracy: 0.9781 - val_loss: 0.0807 - val_accuracy: 0.9758
Epoch 

In [38]:
result = model.evaluate(x_test, y_test, verbose = 1)

313/313 [==============================] - 1s 2ms/step - loss: 0.0748 - accuracy: 0.9803


In [40]:
print('test loss, test acc:', result)

test loss, test acc: [0.07475697994232178, 0.9803000092506409]


##학습이 어떤식으로 이루어져서 정답을 맞추는지가 직관적으로 이해되지 않아 서술해봄

In [52]:
#x_trian 샘플은 60000개가 있음. 각 샘플마다의 정답이 레이블 되어 있음
#예를 들어 x_train[59999] :6만번째 샘플(그림)의 정답은 8이다
x_train[59999]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [53]:
y_train[59999]

8

**앞서 구성한 다층퍼셉트론 네트워크에서 x_train[59999]가 가지고 있는 28*28 = 784개의 인풋값을 전부 계산해서 내린 결론의 값은 숫자 8 이미지이구나! 라고 학습을 한다. 이런 작업을 x_train[0]~[59999]까지 전부 머신이 수행한다. 그렇게 학습된 머신은 새로운 테스트 데이터셋에 적용시켜서 숫자 그림을 잘 맞추는지 아닌지 테스트 하는 것이 MLP_mnist의 목표**